In [1]:
%%bash
source /opt/conda/bin/activate opensfm
mamba install -y 'numpy>=1.19,<2.0.0'
export PATH=$PATH:/opt/utils/OpenSfM/bin
# Take initial guess of intrinsic parameters through metadata
opensfm extract_metadata turtle/

# Detect features points 
opensfm detect_features turtle/

# Match feature points across images
opensfm match_features turtle/

# This creates "tracks" for the features. That is to say, if a feature in image 1 is matched with one in image 2,
# and in turn that one is matched with one in image 3, then it links the matches between 1 and 3. In this case, 
# it does not matter since we only have two images
opensfm create_tracks turtle/

# Calculates the essential matrix, the camera pose and the reconstructed feature points
opensfm reconstruct turtle/

# For visualization using Open3D
opensfm export_ply turtle/

Transaction

  Prefix: /opt/conda/envs/opensfm

  All requested packages already installed


Looking for: ["numpy[version='>=1.19,<2.0.0']"]


Pinned packages:
  - python 3.10.*




2024-07-29 03:06:04,591 INFO: Loading existing EXIF for IMG_3191.JPG
2024-07-29 03:06:04,608 INFO: Loading existing EXIF for IMG_3197.JPG
2024-07-29 03:06:04,608 INFO: Loading existing EXIF for IMG_3184.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3203.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3210.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3213.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3217.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3206.JPG
2024-07-29 03:06:04,609 INFO: Loading existing EXIF for IMG_3201.JPG
2024-07-29 03:06:04,610 INFO: Loading existing EXIF for IMG_3200.JPG
2024-07-29 03:06:04,610 INFO: Loading existing EXIF for IMG_3187.JPG
2024-07-29 03:06:04,610 INFO: Loading existing EXIF for IMG_3209.JPG
2024-07-29 03:06:04,610 INFO: Loading existing EXIF for IMG_3189.JPG
2024-07-29 03:06:04,610 INFO: Loading existing EXIF for IMG_3193.JPG
2024-07-29 03:06:04,610 INFO: Load

In [8]:
## RUN THIS CELL ONCE ONLY, you just need to create the Open3D environment once

## You need to select the "Open3D" environment before you run the next cells (top right corner to change kernel)
"""
%%bash
# SWITCH to Open3D
source /opt/conda/bin/activate open3d
python -m ipykernel install --user --name open3d --display-name "Open3D"
"""


'\n%%bash\n# SWITCH to Open3D\nsource /opt/conda/bin/activate open3d\npython -m ipykernel install --user --name open3d --display-name "Open3D"\n'

In [2]:
def get_colors_from_ply(plyfile):
    '''
    Function to read the colors from a .ply file
    returns an RGB array scaled between [0,1]
    '''
    with open(plyfile, 'r') as f:
        colors = []
        columns = {}
        col_idx = 0
        header_done = False
        for line in f.readlines():
            if line.startswith('property'):
                columns[line.split()[-1]] = col_idx
                col_idx += 1
            if header_done:
                line_list = line.split()
                colors.append([float(line_list[columns['red']]),
                              float(line_list[columns['green']]),
                              float(line_list[columns['blue']])
                              ])
            if line.startswith('end_header'):
                header_done = True
        colors = np.array(colors)/255
    return colors
        

In [3]:
import open3d as o3d
import plotly
import plotly.graph_objects as go
import numpy as np
import plotly.graph_objects as go

#-------------------------------------------------------------------
#-------------------------------------------------------------------
#----------------------- Define your folder here -------------------
myfolder = "turtle"
#-------------------------------------------------------------------
#-------------------------------------------------------------------
# read point cloud data
pcd = o3d.io.read_point_cloud(myfolder + "/reconstruction.ply", format='ply')
# store the colors
pcd.colors = o3d.utility.Vector3dVector(get_colors_from_ply(myfolder + '/reconstruction.ply'))

# convert to array
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2], 
            mode='markers',
            marker=dict(size=5, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.write_html('vis-sparse33.html')


OpenSfM has various functions to refine the 3d reconstruction.
See https://opensfm.readthedocs.io/en/latest/using.html for more details.

We will be using the `undistort` and `compute_depthmaps` functions to create a dense reconstruction of the scene

In [5]:
%%bash
source /opt/conda/bin/activate opensfm
mamba install -y 'numpy>=1.19,<2.0.0'
export PATH=$PATH:/opt/utils/OpenSfM/bin

opensfm undistort turtle/
opensfm compute_depthmaps turtle/

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
Transaction

  Prefix: /opt/conda/envs/opensfm

  All requested packages already installed


Looking for: ["numpy[version='>=1.19,<2.0.0']"]


Pinned packages:
  - python 3.10.*




2024-07-29 03:20:24,133 DEBUG: Undistorting the reconstruction
2024-07-29 03:20:25,630 INFO: Undistorting in parallel with 1 processes (141.09765625 MB per image)
2024-07-29 03:20:25,632 DEBUG: Undistorting image IMG_3184.JPG
2024-07-29 03:20:28,169 DEBUG: Undistorting image IMG_3216.JPG
2024-07-29 03:20:30,277 DEBUG: Undistorting image IMG_3182.JPG
2024-07-29 03:20:32,582 DEBUG: Undistorting image IMG_3205.JPG
2024-07-29 03:20:34,922 DEBUG: Undistorting image IMG_3212.JPG
2024-07-29 03:20:37,017 DEBUG: Undistorting image IMG_3179.JPG
2024-07-29 03:20:39,176 DEBUG: Undistorting image IMG_3201.JPG
2024-07-29 03:20:41,330 DEBUG: Undistorting image IMG_3191.JPG
2024-07-29 03:20:43,338 DEBUG: Undistorting image IMG_3188.JPG
2024-07-29 03:20:45,499 DEBUG: Undistorting image IMG_3192.JPG
2024-07-29 03:20:47,710 DEBUG: Undistorting image IMG_3194.JPG
2024-07-29 03:20:49,710 DEBUG: Undistorting image IMG_3203.JPG
2024-07-29 03:20:51,796 DEBUG: Undistorting image IMG_3215.JPG
2024-07-29 03:20:5

In [6]:
pcd_dense = o3d.io.read_point_cloud(myfolder + '/undistorted/depthmaps/merged.ply', format='ply')
pcd_dense.colors = o3d.utility.Vector3dVector(get_colors_from_ply(myfolder + "/undistorted/depthmaps/merged.ply"))
points_dense = np.asarray(pcd_dense.points)
colors_dense = np.asarray(pcd_dense.colors)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points_dense[:,0], y=points_dense[:,1], z=points_dense[:,2], 
            mode='markers',
            marker=dict(size=1, color=colors_dense)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.write_html('vis-dense33.html')